In [2]:
from glob import glob
import xarray as xr
import pandas as pd
import numpy as np
from pyresample import geometry, create_area_def
from satpy import Scene
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Baseline Combined File

In [11]:
period = 'GOM21FA'
extent = (-93.2, 27.6, -87.6, 31)
savefile = '/home/hboi-ouri/Projects/RS_Files/Task2/Cruise/'+ period +'_MODIS_L2.nc'


files = glob('/home/hboi-ouri/Projects/RS_Files/MODISGrids/Cruise/'+ period +'/AQUA_MODIS.*nc')
files.sort()

# Functions to be used

def convtoxr(scn, var):
    #convert from dask array
    data = scn[var].load()
    data = data.where(data >= 0)
    data.name = var
    del(data.attrs['_satpy_id']) ### removing, we don't need this.
    data.attrs['area'] = str(data.attrs['area']) ### We may or may not need this in the future so converting it to a string
    return data

def load_satdata(path, 
                 area_deff=None,
                 res=None,
                 name="GOMEX", proj={"proj": "laea"}, area_extent=extent,
                ):
#   get data and attributes
    data = xr.open_dataset(path, group='geophysical_data')
    navigation = xr.open_dataset(path, group='navigation_data')
    navigation = navigation.rename({'pixel_control_points': 'pixels_per_line'})
    attrs = xr.open_dataset(path).attrs
    
#   area definition if not defined
    if res is None:
        # get the resolution of the Sat product
        val, unit = attrs['spatialResolution'].split()
        res  = xr.DataArray(float(val), attrs={"units": unit})
    if area_deff is None:
        area_deff = create_area_def(name, proj, area_extent=area_extent, units="degrees", resolution=res)
        
#   reprojecting
    scn = Scene()
    swath_def = geometry.SwathDefinition(lons=navigation.longitude, lats= navigation.latitude)
    vnames = [v for v in data.data_vars if 'Rrs' in v]
    for var in vnames:
        scn[var] = data[var]
        scn[var].attrs["area"] = swath_def
    resampled_scn = scn.resample(area_deff, radius_of_influence=int(res))
    resampled_scn.compute()
    lons, lats = resampled_scn[vnames[0]].attrs['area'].get_lonlats()
#   Converting back to xarray dataset
    ds = xr.merge([convtoxr(resampled_scn, var) for var in vnames])
    ds.attrs = attrs
    ds = ds.assign_coords({
                            'lon': (('y', 'x'), lons),
                            'lat': (('y', 'x'), lats),
                            'time': pd.to_datetime(attrs['time_coverage_start'])
                            })
#   getting rid of the crs coord
    ds = ds.drop('crs')
    print(str(ds.time.data))
    return ds

# Area setup for GOMEX
resval = 1000 # resolution of the sat product (300 m for Sentinel and MERIS, 1 km for MODIS)
res = xr.DataArray(resval, attrs={"units": "meters"})
lat_0, lon_0 = 28.5, -90.8 # just some rounded up mean from the lat lon range of the sat product
proj = {'proj': 'laea', 'lat_0': lat_0, 'lon_0': lon_0, 'a': 6371228.0, 'units': 'm'} # projection object
area = create_area_def("GOMEX", proj,
                       area_extent=extent, units="degrees", 
                       resolution=res
                      ) # creating a fixed area to reproject to

# Reprojectig files
dsets = [load_satdata(f, area_deff=area, res=res) for f in files] # looping trough the files
combined = xr.concat(dsets, dim='time') #combining in one file


#save combined file
# give a sensible name i.e. cruiseID_satID.nc for every cruise and satelite
combined.to_netcdf(savefile) 

    
    

Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when conver

2021-10-22T18:15:00.881000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2021-10-22T19:55:00.913000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

...

2021-11-12T18:35:01.458000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2021-11-12T20:10:01.636000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2021-11-13T19:15:01.050000000


# Adding Variables to Baseline Combined File

In [18]:
def get_CDOM300(ds, lambd=300):

        ds=ds.where(ds>0)
        
        alpha = -.0206
        beta = -.6128
        gamma = -.007
        delta = -.4944
        epsilon = .9362
        zeta = .9666

        aCDOM300 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return aCDOM300
    
    
def get_CDOM355(ds, lambd=355):
        
        ds=ds.where(ds>0)

        alpha = .0376
        beta = -.8714
        gamma = -.0352
        delta = -.2739
        epsilon = .9591
        zeta = -.1071

        aCDOM355 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return aCDOM355
    
    
def get_S275_295(ds):
        
        ds=ds.where(ds>0)

        alpha = -.0537
        beta = .2689
        gamma = .1017
        delta = -.2097
        epsilon = -.0893
        zeta = -3.6853

        S275_295 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return S275_295


def get_DOC(ds):
        
        ds=ds.where(ds>0)
        
        alpha = -.0206
        beta = -.6128
        gamma = -.007
        delta = -.4944
        epsilon = .9362
        zeta = .9666

        old_aCDOM300 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)
        
        alpha = -.0537
        beta = .2689
        gamma = .1017
        delta = -.2097
        epsilon = -.0893
        zeta = -3.6853

        S275_295 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)
        
        DOC = (old_aCDOM300)/(np.exp(-15.05 - 33.95 * S275_295) + 
                            np.exp(-1.502 - 104.3 * S275_295))

        return DOC
    
    

    

In [19]:
####For Cruises:

combfiles = glob('/home/hboi-ouri/Projects/RS_Files/Task2/Cruise/*_MODIS_L2.nc')
combfiles.sort()

for file in combfiles:

    cruise = file.split('/')[7][0:7]
    
    if cruise == 'GOM21SU':
        index = '1'
    elif cruise == 'GOM21FA':
        index = '2'
    elif cruise == 'GOM22SP':
        index = '3'
    elif cruise == 'GOM22SU':
        index = '4'
        
    with xr.open_dataset(file) as ds:
        data = ds.load()
        data['aCDOM300'] = get_CDOM300(ds)
        data['aCDOM355'] = get_CDOM355(ds)
        data['S275_295'] = get_S275_295(ds)
        data['DOC'] = get_DOC(ds)
        
    
        
        data.to_netcdf('/home/hboi-ouri/Projects/RS_Files/Task2/CruiseaCDOM/'+ index + cruise +'_MODIS_L2_aCDOM.nc') 

    
    

In [5]:
####For Monthly:
combfiles = glob('/home/hboi-ouri/Projects/RS_Files/Task4/*_MODIS_L2.nc')
combfiles.sort()

for file in combfiles:

    month = file.split('/')[6][0:7]
        
    with xr.open_dataset(file) as ds:
        data = ds.load()
        data['aCDOM300'] = get_CDOM300(ds)
        data['aCDOM355'] = get_CDOM355(ds)
        data['S275_295'] = get_S275_295(ds)
        data['DOC'] = get_DOC(ds)
        data['aCDOM300_special'] = get_aCDOM300_special(ds)
        
    
        
        data.to_netcdf('/home/hboi-ouri/Projects/RS_Files/Task4/'+ month +'_MODIS_L2_aCDOM.nc')

    

# For Task 8:

In [20]:
period = 'GOM22SU'
extent = (-93.2, 27.6, -87.6, 31)

files = glob('/home/hboi-ouri/Projects/RS_Files/MODISGrids/Cruise/'+ period +'/AQUA_MODIS.*nc')
files.sort()



# Functions to be used

for file in files:

    def convtoxr(scn, var):
        #convert from dask array
        data = scn[var].load()
        data = data.where(data >= 0)
        data.name = var
        del(data.attrs['_satpy_id']) ### removing, we don't need this.
        data.attrs['area'] = str(data.attrs['area']) ### We may or may not need this in the future so converting it to a string
        return data

    def load_satdata(path, 
                     area_deff=None,
                     res=None,
                     name="GOMEX", proj={"proj": "laea"}, area_extent=extent,
                    ):
    #   get data and attributes
        data = xr.open_dataset(path, group='geophysical_data')
        navigation = xr.open_dataset(path, group='navigation_data')
        navigation = navigation.rename({'pixel_control_points': 'pixels_per_line'})
        attrs = xr.open_dataset(path).attrs

    #   area definition if not defined
        if res is None:
            # get the resolution of the Sat product
            val, unit = attrs['spatialResolution'].split()
            res  = xr.DataArray(float(val), attrs={"units": unit})
        if area_deff is None:
            area_deff = create_area_def(name, proj, area_extent=area_extent, units="degrees", resolution=res)

    #   reprojecting
        scn = Scene()
        swath_def = geometry.SwathDefinition(lons=navigation.longitude, lats= navigation.latitude)
        vnames = [v for v in data.data_vars if 'Rrs' in v]
        for var in vnames:
            scn[var] = data[var]
            scn[var].attrs["area"] = swath_def
        resampled_scn = scn.resample(area_deff, radius_of_influence=int(res))
        resampled_scn.compute()
        lons, lats = resampled_scn[vnames[0]].attrs['area'].get_lonlats()
    #   Converting back to xarray dataset
        ds = xr.merge([convtoxr(resampled_scn, var) for var in vnames])
        ds.attrs = attrs
        ds = ds.assign_coords({
                                'lon': (('y', 'x'), lons),
                                'lat': (('y', 'x'), lats),
                                'time': pd.to_datetime(attrs['time_coverage_start'])
                                })
    #   getting rid of the crs coord
        ds = ds.drop('crs')
        print(str(ds.time.data))
        return ds

    # Area setup for GOMEX
    resval = 1000 # resolution of the sat product (300 m for Sentinel and MERIS, 1 km for MODIS)
    res = xr.DataArray(resval, attrs={"units": "meters"})
    lat_0, lon_0 = 28.5, -90.8 # just some rounded up mean from the lat lon range of the sat product
    proj = {'proj': 'laea', 'lat_0': lat_0, 'lon_0': lon_0, 'a': 6371228.0, 'units': 'm'} # projection object
    area = create_area_def("GOMEX", proj,
                           area_extent=extent, units="degrees", 
                           resolution=res
                          ) # creating a fixed area to reproject to

    # Reprojectig files
    dsets = load_satdata(file, area_deff=area, res=res) # looping trough the files
#     combined = xr.concat(dsets, dim='time') #combining in one file


    #save combined file
    # give a sensible name i.e. cruiseID_satID.nc for every cruise and satelite
    
    dsets.to_netcdf('/home/hboi-ouri/Projects/RS_Files/Task8/NC Files/'+ file.split('.')[1][:15] +'_MODIS_L2.nc') 

    
    

Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when conver

2022-07-06T18:40:01.366000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2022-07-06T18:45:01.220000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

...

/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2022-07-27T18:55:01.881000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2022-07-28T19:40:01.312000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

In [21]:
def get_CDOM300(ds, lambd=300):

        ds=ds.where(ds>0)
        
        alpha = -.0206
        beta = -.6128
        gamma = -.007
        delta = -.4944
        epsilon = .9362
        zeta = .9666

        aCDOM300 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return aCDOM300
    
    
def get_CDOM355(ds, lambd=355):
        
        ds=ds.where(ds>0)

        alpha = .0376
        beta = -.8714
        gamma = -.0352
        delta = -.2739
        epsilon = .9591
        zeta = -.1071

        aCDOM355 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return aCDOM355
    
    
def get_S275_295(ds):
        
        ds=ds.where(ds>0)

        alpha = -.0537
        beta = .2689
        gamma = .1017
        delta = -.2097
        epsilon = -.0893
        zeta = -3.6853

        S275_295 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)

        return S275_295


def get_DOC(ds):
        
        ds=ds.where(ds>0)
        
        alpha = -.0206
        beta = -.6128
        gamma = -.007
        delta = -.4944
        epsilon = .9362
        zeta = .9666

        old_aCDOM300 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)
        
        alpha = -.0537
        beta = .2689
        gamma = .1017
        delta = -.2097
        epsilon = -.0893
        zeta = -3.6853

        S275_295 = np.exp(alpha * np.log(ds.Rrs_443.where(ds.Rrs_443>=0)) +
                      beta * np.log(ds.Rrs_488.where(ds.Rrs_488>=0)) +
                      gamma * np.log(ds.Rrs_531.where(ds.Rrs_531>=0)) +
                      delta * np.log(ds.Rrs_555.where(ds.Rrs_555>=0)) +
                      epsilon * np.log(ds.Rrs_667.where(ds.Rrs_667>=0)) + zeta)
        
        DOC = (old_aCDOM300)/(np.exp(-15.05 - 33.95 * S275_295) + 
                            np.exp(-1.502 - 104.3 * S275_295))

        return DOC
    
    

    

In [28]:
####For Monthly:
files = glob('/home/hboi-ouri/Projects/RS_Files/Task8/NC Files/*.nc')
files.sort()

for file in files:
        
    with xr.open_dataset(file) as ds:
        data = ds.load()
        data['aCDOM300'] = get_CDOM300(ds)
        data['aCDOM355'] = get_CDOM355(ds)
        data['S275_295'] = get_S275_295(ds)
        data['DOC'] = get_DOC(ds)
        
    
        
        data.to_netcdf('/home/hboi-ouri/Projects/RS_Files/Task8/Variable Files/'+ file.split('/')[7][:15] +'_MODIS_L2.nc') 


# For Chlorophyll

In [4]:
extent = (-93.2, 27.6, -87.6, 31)

months = []
monthfiles = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/Raw/*')
monthfiles.sort()
for period in monthfiles:
    months.append(period[50:57])

for month in months:
    # Functions to be used
    files = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/Raw/'+ month +'/*AQUA_MODIS.*nc')
    files.sort()

    def convtoxr(scn, var):
        #convert from dask array
        data = scn[var].load()
        data = data.where(data >= 0)
        data.name = var
        del(data.attrs['_satpy_id']) ### removing, we don't need this.
        data.attrs['area'] = str(data.attrs['area']) ### We may or may not need this in the future so converting it to a string
        return data

    def load_satdata(path, 
                     area_deff=None,
                     res=None,
                     name="GOMEX", proj={"proj": "laea"}, area_extent=extent,
                    ):
    #   get data and attributes
        data = xr.open_dataset(path, group='geophysical_data')
        navigation = xr.open_dataset(path, group='navigation_data')
        navigation = navigation.rename({'pixel_control_points': 'pixels_per_line'})
        attrs = xr.open_dataset(path).attrs

    #   area definition if not defined
        if res is None:
            # get the resolution of the Sat product
            val, unit = attrs['spatialResolution'].split()
            res  = xr.DataArray(float(val), attrs={"units": unit})
        if area_deff is None:
            area_deff = create_area_def(name, proj, area_extent=area_extent, units="degrees", resolution=res)

    #   reprojecting
        scn = Scene()
        swath_def = geometry.SwathDefinition(lons=navigation.longitude, lats= navigation.latitude)
        vnames = [v for v in data.data_vars if 'chlor_a' in v]
        for var in vnames:
            scn[var] = data[var]
            scn[var].attrs["area"] = swath_def
        resampled_scn = scn.resample(area_deff, radius_of_influence=int(res))
        resampled_scn.compute()
        lons, lats = resampled_scn[vnames[0]].attrs['area'].get_lonlats()
    #   Converting back to xarray dataset
        ds = xr.merge([convtoxr(resampled_scn, var) for var in vnames])
        ds.attrs = attrs
        ds = ds.assign_coords({
                                'lon': (('y', 'x'), lons),
                                'lat': (('y', 'x'), lats),
                                'time': pd.to_datetime(attrs['time_coverage_start'])
                                })
    #   getting rid of the crs coord
        ds = ds.drop('crs')
        print(str(ds.time.data))
        return ds

    # Area setup for GOMEX
    resval = 1000 # resolution of the sat product (300 m for Sentinel and MERIS, 1 km for MODIS)
    res = xr.DataArray(resval, attrs={"units": "meters"})
    lat_0, lon_0 = 28.5, -90.8 # just some rounded up mean from the lat lon range of the sat product
    proj = {'proj': 'laea', 'lat_0': lat_0, 'lon_0': lon_0, 'a': 6371228.0, 'units': 'm'} # projection object
    area = create_area_def("GOMEX", proj,
                           area_extent=extent, units="degrees", 
                           resolution=res
                          ) # creating a fixed area to reproject to

    # Reprojectig files
    dsets = [load_satdata(f, area_deff=area, res=res) for f in files] # looping trough the files
    combined = xr.concat(dsets, dim='time') #combining in one file


    #save combined file
    # give a sensible name i.e. cruiseID_satID.nc for every cruise and satelite

    savefile = '/home/hboi-ouri/Projects/RS_Files/Chlorophyll/MonthCombined/'+ month +'_MODIS_L2.nc'
    combined.to_netcdf(savefile) 

    
    

Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when conver

2020-01-01T18:40:00.750000000
2020-01-01T20:15:00.928000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2020-01-01T20:20:00.782000000
2020-01-02T19:20:01.819000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

...

2022-12-29T19:25:01.551000000
2022-12-30T18:30:00.974000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2022-12-30T20:10:01.007000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


2022-12-31T19:15:01.906000000


# For Chlorophyll Cruise

In [6]:
extent = (-93.2, 27.6, -87.6, 31)

months = []
monthfiles = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/*')
monthfiles.sort()
for period in monthfiles:
    months.append(period[56:62])

for month in months:
    # Functions to be used
    files = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/'+ month +'/*AQUA_MODIS.*nc')
    files.sort()

    def convtoxr(scn, var):
        #convert from dask array
        data = scn[var].load()
        data = data.where(data >= 0)
        data.name = var
        del(data.attrs['_satpy_id']) ### removing, we don't need this.
        data.attrs['area'] = str(data.attrs['area']) ### We may or may not need this in the future so converting it to a string
        return data

    def load_satdata(path, 
                     area_deff=None,
                     res=None,
                     name="GOMEX", proj={"proj": "laea"}, area_extent=extent,
                    ):
    #   get data and attributes
        data = xr.open_dataset(path, group='geophysical_data')
        navigation = xr.open_dataset(path, group='navigation_data')
        navigation = navigation.rename({'pixel_control_points': 'pixels_per_line'})
        attrs = xr.open_dataset(path).attrs

    #   area definition if not defined
        if res is None:
            # get the resolution of the Sat product
            val, unit = attrs['spatialResolution'].split()
            res  = xr.DataArray(float(val), attrs={"units": unit})
        if area_deff is None:
            area_deff = create_area_def(name, proj, area_extent=area_extent, units="degrees", resolution=res)

    #   reprojecting
        scn = Scene()
        swath_def = geometry.SwathDefinition(lons=navigation.longitude, lats= navigation.latitude)
        vnames = [v for v in data.data_vars if 'chlor_a' in v]
        for var in vnames:
            scn[var] = data[var]
            scn[var].attrs["area"] = swath_def
        resampled_scn = scn.resample(area_deff, radius_of_influence=int(res))
        resampled_scn.compute()
        lons, lats = resampled_scn[vnames[0]].attrs['area'].get_lonlats()
    #   Converting back to xarray dataset
        ds = xr.merge([convtoxr(resampled_scn, var) for var in vnames])
        ds.attrs = attrs
        ds = ds.assign_coords({
                                'lon': (('y', 'x'), lons),
                                'lat': (('y', 'x'), lats),
                                'time': pd.to_datetime(attrs['time_coverage_start'])
                                })
    #   getting rid of the crs coord
        ds = ds.drop('crs')
        print(str(ds.time.data))
        return ds

    # Area setup for GOMEX
    resval = 1000 # resolution of the sat product (300 m for Sentinel and MERIS, 1 km for MODIS)
    res = xr.DataArray(resval, attrs={"units": "meters"})
    lat_0, lon_0 = 28.5, -90.8 # just some rounded up mean from the lat lon range of the sat product
    proj = {'proj': 'laea', 'lat_0': lat_0, 'lon_0': lon_0, 'a': 6371228.0, 'units': 'm'} # projection object
    area = create_area_def("GOMEX", proj,
                           area_extent=extent, units="degrees", 
                           resolution=res
                          ) # creating a fixed area to reproject to

    # Reprojectig files
    dsets = [load_satdata(f, area_deff=area, res=res) for f in files] # looping trough the files
    combined = xr.concat(dsets, dim='time') #combining in one file


    #save combined file
    # give a sensible name i.e. cruiseID_satID.nc for every cruise and satelite

    savefile = '/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseCombined/'+ month +'_MODIS_L2.nc'
    combined.to_netcdf(savefile) 

    
    

Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when conver

2021-10-22T18:15:00.881000000
2021-10-22T19:55:00.913000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2021-10-23T18:55:01.944000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

...

2022-07-24T20:05:00.549000000
2022-07-25T19:10:01.427000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2022-07-26T19:50:01.005000000
2022-07-27T18:55:01.881000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2022-07-28T19:40:01.312000000


# For Chlorophyll Cruise Dailies

In [18]:

        
months = []
monthfiles = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/*')
monthfiles.sort()
for period in monthfiles:
    months.append(period[56:62])

    for month in months:
        days = []
        dayfiles = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/'+ month +'/*')
        dayfiles.sort()
        for day in dayfiles:
            if day[74:82] in days:
                continue
            else:
                days.append(day[74:82])
    
    for day in days:
        # Functions to be used
        files = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/'+ month +'/AQUA_MODIS.'+ day +'*nc')
        files.sort()
        print(files)

['/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211022T181500.L2.OC.x.nc', '/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211022T195500.L2.OC.x.nc']
['/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211023T185501.L2.OC.x.nc', '/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211023T190001.L2.OC.x.nc']
['/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211024T194001.L2.OC.x.nc']
['/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211025T184500.L2.OC.x.nc']
['/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211026T193001.L2.OC.x.nc']
['/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211027T183501.L2.OC.x.nc', '/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/FA2021/AQUA_MODIS.20211027T201001.L2.OC.x.nc']
['/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/

In [19]:
extent = (-93.2, 27.6, -87.6, 31)

months = []
monthfiles = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/*')
monthfiles.sort()
for period in monthfiles:
    months.append(period[56:62])

for month in months:
    days = []
    dayfiles = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/'+ month +'/*')
    dayfiles.sort()
    for day in dayfiles:
        if day[74:82] in days:
            continue
        else:
            days.append(day[74:82])
    
    for day in days:
        # Functions to be used
        files = glob('/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseRaw/'+ month +'/AQUA_MODIS.'+ day +'*nc')
        files.sort()

        def convtoxr(scn, var):
            #convert from dask array
            data = scn[var].load()
            data = data.where(data >= 0)
            data.name = var
            del(data.attrs['_satpy_id']) ### removing, we don't need this.
            data.attrs['area'] = str(data.attrs['area']) ### We may or may not need this in the future so converting it to a string
            return data

        def load_satdata(path, 
                         area_deff=None,
                         res=None,
                         name="GOMEX", proj={"proj": "laea"}, area_extent=extent,
                        ):
        #   get data and attributes
            data = xr.open_dataset(path, group='geophysical_data')
            navigation = xr.open_dataset(path, group='navigation_data')
            navigation = navigation.rename({'pixel_control_points': 'pixels_per_line'})
            attrs = xr.open_dataset(path).attrs

        #   area definition if not defined
            if res is None:
                # get the resolution of the Sat product
                val, unit = attrs['spatialResolution'].split()
                res  = xr.DataArray(float(val), attrs={"units": unit})
            if area_deff is None:
                area_deff = create_area_def(name, proj, area_extent=area_extent, units="degrees", resolution=res)

        #   reprojecting
            scn = Scene()
            swath_def = geometry.SwathDefinition(lons=navigation.longitude, lats= navigation.latitude)
            vnames = [v for v in data.data_vars if 'chlor_a' in v]
            for var in vnames:
                scn[var] = data[var]
                scn[var].attrs["area"] = swath_def
            resampled_scn = scn.resample(area_deff, radius_of_influence=int(res))
            resampled_scn.compute()
            lons, lats = resampled_scn[vnames[0]].attrs['area'].get_lonlats()
        #   Converting back to xarray dataset
            ds = xr.merge([convtoxr(resampled_scn, var) for var in vnames])
            ds.attrs = attrs
            ds = ds.assign_coords({
                                    'lon': (('y', 'x'), lons),
                                    'lat': (('y', 'x'), lats),
                                    'time': pd.to_datetime(attrs['time_coverage_start'])
                                    })
        #   getting rid of the crs coord
            ds = ds.drop('crs')
            print(str(ds.time.data))
            return ds

        # Area setup for GOMEX
        resval = 1000 # resolution of the sat product (300 m for Sentinel and MERIS, 1 km for MODIS)
        res = xr.DataArray(resval, attrs={"units": "meters"})
        lat_0, lon_0 = 28.5, -90.8 # just some rounded up mean from the lat lon range of the sat product
        proj = {'proj': 'laea', 'lat_0': lat_0, 'lon_0': lon_0, 'a': 6371228.0, 'units': 'm'} # projection object
        area = create_area_def("GOMEX", proj,
                               area_extent=extent, units="degrees", 
                               resolution=res
                              ) # creating a fixed area to reproject to

        # Reprojectig files
        dsets = [load_satdata(f, area_deff=area, res=res) for f in files] # looping trough the files
        combined = xr.concat(dsets, dim='time') #combining in one file


        #save combined file
        # give a sensible name i.e. cruiseID_satID.nc for every cruise and satelite

        savefile = '/home/hboi-ouri/Projects/RS_Files/Chlorophyll/CruiseDailyCombined/'+ month +'/'+ day +'_MODIS_L2.nc'
        combined.to_netcdf(savefile) 

    
    

Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when conver

2021-10-22T18:15:00.881000000
2021-10-22T19:55:00.913000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinat

2021-10-23T18:55:01.944000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


...

/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when conver

2022-07-25T19:10:01.427000000
2022-07-26T19:50:01.005000000


Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
Rounding shape to (380, 542) and resolution from (1000.0, 1000.0) meters to (999.0798276127949, 999.6147938626385) meters
/home/hboi-ouri/miniconda3/envs/Satpy/lib/pyth

2022-07-27T18:55:01.881000000
2022-07-28T19:40:01.312000000


/home/hboi-ouri/miniconda3/envs/Satpy/lib/python3.8/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
